In [ ]:
KOU Big Data Final

Sevde Kaşkaya 150201137

Bu çalışmada Keras kullanarak dataset üzerinde mağaza ve ürün satış bilgileri ile prediction yapmaya çalıştım.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.models import load_model


dataset = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
print (dataset.head())
print (dataset.shape)

train, test = train_test_split(dataset, test_size=0.2, random_state=42)
n_shop = len(dataset.shop_id.unique())
n_item = len(dataset.item_id.unique())

print ("\n\n", "Shop: ", n_shop, "İtem: ", n_item)

Shop ve Item için vektör oluşturuyorum:

In [ ]:
shop_input = Input(shape=[1], name="Shop-Input")
shop_embedding = Embedding(input_dim=150000, output_dim =6, name="Shop-Embedding")(shop_input)
shop_vec = Flatten(name="Flatten-Shop")(shop_embedding)


item_input = Input(shape=[1], name="item-Input")
item_embedding = Embedding(input_dim=150000, output_dim =6,name="item-Embedding")(item_input)
item_vec = Flatten(name="Flatten-item")(item_embedding)

conc = Concatenate()([shop_vec, item_vec])

Layer ekliyorum ve modeli compile ediyorum:

In [ ]:
fc1 = Dense(20, activation='relu')(conc)
fc2 = Dense(10, activation='relu')(fc1)
out = Dense(1)(fc2)


model = Model([item_input, shop_input], out)
model.compile('adam', 'mean_squared_error')

Modeli train ettim:

In [ ]:
  if os.path.exists('regression_model2.h5'):
    model = load_model('regression_model2.h5')
else:
    history = model.fit([train.shop_id, train.item_id], train.item_cnt_day, epochs=5, verbose=1)
    model.save('regression_model2.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

Model predictions:

In [ ]:
model.evaluate([test.shop_id, test.item_id], test.item_cnt_day)
predictions = model.predict([test.shop_id.head(10), test.item_id.head(10)])

print ("\n\n Model predictions: \n")
for i in range(0,10):
	print(predictions[i], test.item_cnt_day.iloc[i])